In [1]:
import ee, pandas as pd

In [2]:
# configurations
CSV_PATH      = "ring_width_output/tree_ring_coordinates.csv"
PROJECT_ID    = "treegrowth-471820"
BASE_TEMP_C   = 5.0
START_YEAR    = 1950
END_YEAR      = 2025
SAMPLE_SCALE  = 10000         # meters (fine for ERA5-Land ~9 km)
EXPORT_TO     = "DRIVE"       # "DRIVE" or "ASSET"
DRIVE_FOLDER  = "GEE_Exports" # created if missing

In [3]:
# EE Authorization and intiialization
try:
    ee.Initialize()
except Exception:
    ee.Authenticate()  # follow the link on first run
    ee.Initialize(project=PROJECT_ID)

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


In [4]:
# Load Canada sites
sites_df = pd.read_csv(CSV_PATH)
sites_df = sites_df.dropna(subset=["lat","lon"]).copy()
sites_df["site_id"] = sites_df.get("site_id", pd.Series([f"site_{i:06d}" for i in range(len(sites_df))]))
print(f"Loaded {len(sites_df)} sites")

def _row_to_feat(r):
    geom = ee.Geometry.Point([float(r["lon"]), float(r["lat"])])
    return ee.Feature(geom, {"site_id": str(r["site_id"])})
sites_fc = ee.FeatureCollection([_row_to_feat(r) for _, r in sites_df.iterrows()])

Loaded 595 sites


Muñoz Sabater, J., (2019): ERA5-Land monthly averaged data from 1981 to present. Copernicus Climate Change Service (C3S) Climate Data Store (CDS). (<date of access>), doi:10.24381/cds.68d2bb30

We use the **ERA5-Land (daily aggregates)** collection: `ECMWF/ERA5_LAND/DAILY_AGGR` and the band `temperature_2m` (Kelvin).  
Growing Degree Day (GDD) per day is: `max(0, T(°C) - BASE_TEMP_C)`. We sum daily values per calendar year.

In [5]:
era5 = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR") \
        .filterDate(f"{START_YEAR}-01-01", f"{END_YEAR+1}-01-01")

In [ ]:
# Bands vary by dataset; we'll use 2m air temp (daily mean) and precip total:
# t2m_mean: daily mean 2 m temperature (Kelvin), total_precipitation: daily sum (m)
# Convert K->C and precip m->mm

first_img = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR") \
                .filterDate(f"{START_YEAR}-01-01", f"{START_YEAR}-12-31") \
                .first()
print(first_img.bandNames().getInfo())


def add_derived_daily(img):
    # Daily mean 2m temp (Kelvin) -> °C
    t2c = img.select("temperature_2m").subtract(273.15).rename("t2m_c")
    # Daily precip sum (meters) -> mm
    pr_mm = img.select("total_precipitation_sum").multiply(1000.0).rename("pr_mm")
    # Daily GDD contribution = max(0, Tmean - base)
    gdd_day = t2c.subtract(BASE_TEMP_C).max(0).rename("gdd_day_c")
    return img.addBands([t2c, pr_mm, gdd_day])

daily = era5.map(add_derived_daily)


['dewpoint_temperature_2m', 'temperature_2m', 'skin_temperature', 'soil_temperature_level_1', 'soil_temperature_level_2', 'soil_temperature_level_3', 'soil_temperature_level_4', 'lake_bottom_temperature', 'lake_ice_depth', 'lake_ice_temperature', 'lake_mix_layer_depth', 'lake_mix_layer_temperature', 'lake_shape_factor', 'lake_total_layer_temperature', 'snow_albedo', 'snow_cover', 'snow_density', 'snow_depth', 'snow_depth_water_equivalent', 'snowfall_sum', 'snowmelt_sum', 'temperature_of_snow_layer', 'skin_reservoir_content', 'volumetric_soil_water_layer_1', 'volumetric_soil_water_layer_2', 'volumetric_soil_water_layer_3', 'volumetric_soil_water_layer_4', 'forecast_albedo', 'surface_latent_heat_flux_sum', 'surface_net_solar_radiation_sum', 'surface_net_thermal_radiation_sum', 'surface_sensible_heat_flux_sum', 'surface_solar_radiation_downwards_sum', 'surface_thermal_radiation_downwards_sum', 'evaporation_from_bare_soil_sum', 'evaporation_from_open_water_surfaces_excluding_oceans_sum', '

In [7]:
# Aggregrate by year (per-site)
years = list(range(START_YEAR, END_YEAR + 1))

def annual_stats_for_year(y):
    yearly = daily.filterDate(f"{y}-01-01", f"{y+1}-01-01")
    # Reduce the daily stack into annual sums/means
    annual_sum = yearly.select(["gdd_day_c","pr_mm"]).sum() \
        .rename(["gdd_sum_c","pr_sum_mm"])
    annual_mean = yearly.select(["t2m_c"]).mean().rename(["t2m_mean_c"])
    annual_img = annual_sum.addBands(annual_mean) \
        .set({"year": y})

    # Sample at point locations
    sampled = annual_img.sampleRegions(
        collection=sites_fc,
        scale=SAMPLE_SCALE,
        geometries=False
    )
    # Attach year property to each feature
    return sampled.map(lambda f: f.set({"year": y}))

fc_list = [annual_stats_for_year(y) for y in years]
annual_fc = ee.FeatureCollection(fc_list).flatten()

In [8]:
task = ee.batch.Export.table.toDrive(
    collection=annual_fc,
    description=f"CAN_sites_ERA5Land_annual_{START_YEAR}_{END_YEAR}",
    fileFormat="CSV",
    folder=DRIVE_FOLDER
)
task.start()
print("Started Drive export:", task.id)

Started Drive export: AVHDXASM7YU7HMWBWGN55AHQ
